# Recomendation System


### Collaborative Filtering - utilizará "conhecimento de multidão"

In [3]:
# Inicializando Spark
import findspark
findspark.init('/home/macaubas/spark-3.2.1-bin-hadoop3.2')
import pyspark

from pyspark.sql import SparkSession 

spark = SparkSession.builder.appName("recomendationSystem").getOrCreate()

In [4]:
from pyspark.ml.recommendation import ALS

from pyspark.ml.evaluation import RegressionEvaluator

## Carregando dados

In [5]:
data  = spark.read.csv('movielens_ratings.csv', inferSchema=True, header=True)

data.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [6]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [7]:
# Sumarização
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



## Criando modelo

In [8]:
treino, teste = data.randomSplit([0.8, 0.2]) 

In [9]:
als = ALS(maxIter = 5, regParam = 0.01, userCol = 'userId', itemCol = 'movieId', ratingCol='rating')

In [10]:
model = als.fit(treino)

22/05/26 18:06:42 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/05/26 18:06:42 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/05/26 18:06:42 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [12]:
prev = model.transform(teste)

In [13]:
prev.show()

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|      1|   1.0|     3|-1.1536826|
|      1|   1.0|     7| 0.2599645|
|      1|   1.0|    18| -2.355354|
|      1|   1.0|    20| 3.1402643|
|      6|   1.0|    12| 0.6323263|
|      6|   1.0|    17| 2.0762157|
|      6|   1.0|    28| 1.1642405|
|      3|   1.0|     7| 1.7916229|
|      3|   1.0|     9|-0.7752188|
|      3|   2.0|    22|0.19113645|
|      3|   1.0|    26| 0.9200826|
|      3|   1.0|    29|0.42141575|
|      5|   1.0|     5|0.72000194|
|      5|   1.0|     6|0.39580426|
|      5|   1.0|    14| 1.9717283|
|      5|   2.0|    18| 0.3949222|
|      4|   3.0|     2| 1.6600506|
|      4|   2.0|    13|  2.470923|
|      4|   1.0|    14| 1.0897304|
|      7|   1.0|     7|0.22231805|
+-------+------+------+----------+
only showing top 20 rows



### Quando o valor da previsão é negativo, indica que o usuário provavelmente não gostará do filme

## Avaliando o modelo

In [16]:
# Avaliar por raiz quadrada do erro médio
evaluator = RegressionEvaluator(metricName = 'rmse', labelCol='rating', predictionCol='prediction')

In [17]:
rmse = evaluator.evaluate(prev)

In [18]:
print(f"Raíz do erro quadrático médio: \n{rmse:.2}")

Raíz do erro quadrático médio: 
2.0


## Nossa previsão não é boa, isso deve-se ao fato do nosso banco de dados ser pequeno.

## Vendo recomendações para 1 usuário

In [21]:
um_usuario = teste.filter(teste['userId'] == 11).select(['movieId', 'userId'])

In [22]:
um_usuario.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     35|    11|
|     38|    11|
|     39|    11|
|     43|    11|
|     45|    11|
|     48|    11|
|     64|    11|
|     67|    11|
|     72|    11|
|     88|    11|
|     89|    11|
+-------+------+



## Verificando o que recomendaríamos para este usuário

In [23]:
recommendations = model.transform(um_usuario)

In [24]:
recommendations.orderBy('prediction', ascending = False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     64|    11|  6.042179|
|     38|    11|  5.246582|
|     45|    11| 2.5490427|
|     72|    11| 2.5316136|
|     43|    11| 2.5046172|
|     67|    11| 2.4412787|
|     35|    11| 2.3146455|
|     39|    11| 2.2624078|
|     48|    11| 1.9504893|
|      9|    11|  0.630543|
|     88|    11|-1.1304348|
|     89|    11|-1.9932177|
+-------+------+----------+



## Conclusão

Baseado em usuários que assistiram filmes similares ao usuário 11, podemos recomendar os filmes 64 e 38 como prováveis filmes que o usuário 11 gostaria muito de assistir. Os filmes com previsão entre 2.54 e 2.26 o usuário poderia ser indiferente - levando em consideração que nosso RMSE é de 2. Os filmes com previsão menores que 2, é provável que os usuário seja indiferente ou não goste de forma alguma.


O ideal seria aumentar a base de dados de forma a reduzir nosso erro e assim conseguir fazer previsões mais consistentes para o usuário.

Além disso, o modelo é incapaz de lidar com usuário "Cold Start" - aqueles que não assistiram nenhum filme na plataforma ainda. Para lidar com este problema, reocmenda-se um breve questionário para melhor identificar as preferência daquele novo usário.